In [49]:
sc.stop()

In [50]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.{broadcast, split, lit}
import java.sql.Date
val spark = SparkSession.builder().appName("Jupyter").getOrCreate()
//explicitly set the iceberg connection due to connection lost after several minutes
spark.sql("SET spark.sql.catalog.my_catalog.uri=http://192.168.56.1:8181")

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.{broadcast, split, lit}
import java.sql.Date
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6cd297b4
res39: org.apache.spark.sql.DataFrame = [key: string, value: string]


In [27]:
//read the csv files first

val match_detailsBucketed = spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/match_details.csv")

val matchesBucketed = spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/matches.csv")
                        
val me_ma_plBucketed = spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/medals_matches_players.csv")

val medalsBucketed = spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/medals.csv")
                        
val mapsBucketed = spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/maps.csv")

// creating tables and populates them with the csv data

// matches details

val matchDetailsDDL = """
CREATE TABLE IF NOT EXISTS bootcamp.match_details_bucketed (
     match_id STRING,
     player_gamertag STRING,
     player_total_kills INTEGER,
     player_total_deaths INTEGER
 )
 USING iceberg
 PARTITIONED BY (bucket(4, match_id));
"""
 spark.sql(matchDetailsDDL)

 match_detailsBucketed.select(
     $"match_id", $"player_gamertag", $"player_total_kills", $"player_total_deaths")
     .write.mode("append")
   .bucketBy(4, "match_id").saveAsTable("bootcamp.match_details_bucketed")


// matches table
                        
spark.sql("""DROP TABLE IF EXISTS bootcamp.matches_bucketed""")
val matchesDDL = """
CREATE TABLE IF NOT EXISTS bootcamp.matches_bucketed (
    match_id STRING,
    mapid string,
    is_team_game BOOLEAN,
    playlist_id STRING,
    completion_date TIMESTAMP
 )
 USING iceberg
 PARTITIONED BY (completion_date, bucket(4, match_id));
"""

spark.sql(matchesDDL)

// Break large dataset into smaller chunks
val processedData = matchesBucketed
  .select("match_id", "mapid", "is_team_game", "playlist_id", "completion_date")

// Process in batches if extremely large
val distinctDates = processedData.select("completion_date").distinct().collect()

distinctDates.foreach { dateRow =>
  val specificDate = dateRow.getAs[java.sql.Timestamp](0)
  
  processedData
    .filter(col("completion_date") === specificDate)
    .write
    .mode("append")
    .partitionBy("completion_date")
    .bucketBy(4, "match_id")
    .saveAsTable("bootcamp.matches_bucketed")
}



//medal_maps_players table

spark.sql("""DROP TABLE IF EXISTS bootcamp.me_ma_pl_bucketed""")
val memapldDDL = """
CREATE TABLE IF NOT EXISTS bootcamp.me_ma_pl_bucketed (
    match_id STRING,
    player_gamertag STRING,
    medal_id bigint,
    count integer
 )
 USING iceberg
 partitioned by (bucket(4,match_id));
"""

spark.sql(memapldDDL)

me_ma_plBucketed.select(
     $"match_id", $"player_gamertag", $"medal_id", $"count"
     )
     .write.mode("append")
     .bucketBy(4,"match_id")
   .saveAsTable("bootcamp.me_ma_pl_bucketed")


//medals table

spark.sql("""DROP TABLE IF EXISTS bootcamp.medals_bucketed""")
val medalsDDL = """
CREATE TABLE IF NOT EXISTS bootcamp.medals_bucketed (
    medal_id STRING,
    name STRING
 )
 USING iceberg;
"""

spark.sql(medalsDDL)

medalsBucketed.select(
     $"medal_id", $"name"
     )
     .write.mode("append")
   .saveAsTable("bootcamp.medals_bucketed")


//maps table

spark.sql("""DROP TABLE IF EXISTS bootcamp.maps_bucketed""")
val mapsDDL = """
CREATE TABLE IF NOT EXISTS bootcamp.maps_bucketed (
    mapid STRING,
    name STRING
 )
 USING iceberg;
"""

spark.sql(mapsDDL)

mapsBucketed.select(
     $"mapid", $"name"
     )
     .write.mode("append")
   .saveAsTable("bootcamp.maps_bucketed")





match_detailsBucketed: org.apache.spark.sql.DataFrame = [match_id: string, player_gamertag: string ... 34 more fields]
matchesBucketed: org.apache.spark.sql.DataFrame = [match_id: string, mapid: string ... 8 more fields]
me_ma_plBucketed: org.apache.spark.sql.DataFrame = [match_id: string, player_gamertag: string ... 2 more fields]
medalsBucketed: org.apache.spark.sql.DataFrame = [medal_id: bigint, sprite_uri: string ... 10 more fields]
mapsBucketed: org.apache.spark.sql.DataFrame = [mapid: string, name: string ... 1 more field]
matchDetailsDDL: String =
"
CREATE TABLE IF NOT EXISTS bootcamp.match_details_bucketed (
     match_id STRING,
     player_gamertag STRING,
     player_total_kills INTEGER,
     player_total_deaths INTEGER
 )
 USING iceberg
 PARTITIONED BY (bucket(4, match_id));...


In [3]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

val bucketedMatches = spark.table("bootcamp.matches_bucketed").sort("match_id")
val bucketedMatchDetails = spark.table("bootcamp.match_details_bucketed").sort("match_id")

val bucketedJoin = bucketedMatches.as("m")
   .join(bucketedMatchDetails.as("md"), $"m.match_id" === $"md.match_id")
   .select($"m.match_id", $"m.completion_date", $"md.player_gamertag", $"md.player_total_kills", $"md.player_total_deaths")
// .take(5)

bucketedJoin.createOrReplaceTempView("bucketed_result")
spark.sql("SELECT * FROM bucketed_result LIMIT 10").show()

+--------------------+-------------------+---------------+------------------+-------------------+
|            match_id|    completion_date|player_gamertag|player_total_kills|player_total_deaths|
+--------------------+-------------------+---------------+------------------+-------------------+
|00169217-cca6-4b4...|2016-03-13 00:00:00|  King Terror V|                14|                  7|
|00169217-cca6-4b4...|2016-03-13 00:00:00|      King Sope|                11|                  5|
|00169217-cca6-4b4...|2016-03-13 00:00:00|       mcnaeric|                10|                 14|
|00169217-cca6-4b4...|2016-03-13 00:00:00|    EXTREMENOVA|                 8|                 10|
|00169217-cca6-4b4...|2016-03-13 00:00:00| Psych0ticCamel|                 8|                 14|
|00169217-cca6-4b4...|2016-03-13 00:00:00|Trap Lord David|                 8|                 12|
|00169217-cca6-4b4...|2016-03-13 00:00:00|       DJ RAHHH|                13|                 11|
|00169217-cca6-4b4..

bucketedMatches: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [match_id: string, is_team_game: boolean ... 2 more fields]
bucketedMatchDetails: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [match_id: string, player_gamertag: string ... 2 more fields]
bucketedJoin: org.apache.spark.sql.DataFrame = [match_id: string, completion_date: timestamp ... 3 more fields]


In [51]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

val bucketedMatches = spark.table("bootcamp.matches_bucketed").sort("match_id")
val bucketedMatchDetails = spark.table("bootcamp.match_details_bucketed").sort("match_id")
val bucketedMedalMatchPlayers = spark.table("bootcamp.me_ma_pl_bucketed").sort("match_id")
val bucketedMedals = spark.table("bootcamp.medals_bucketed")
val bucketedMaps = spark.table("bootcamp.maps_bucketed")

val bucketedJoin = bucketedMatches.as("m")
   .join(bucketedMatchDetails.as("md"), $"m.match_id" === $"md.match_id")
   .join(bucketedMedalMatchPlayers.as("mmp"), $"md.match_id" === $"mmp.match_id" && $"md.player_gamertag" === $"mmp.player_gamertag")
   .join(broadcast(bucketedMedals).as("me"), $"mmp.medal_id" === $"me.medal_id")
   .join(broadcast(bucketedMaps).as("mp"), $"m.mapid" === $"mp.mapid")
   .select($"m.match_id", $"m.is_team_game", $"m.playlist_id", $"completion_date", 
       $"md.player_gamertag", $"md.player_total_kills",  $"md.player_total_deaths",
        $"mmp.medal_id", $"me.name", $"mmp.count",
        $"mp.mapid", $"mp.name")
//       .explain()

bucketedJoin.createOrReplaceTempView("bucketed_result")
spark.sql("SELECT * FROM bucketed_result LIMIT 3").show()

org.apache.spark.SparkException:  Job aborted due to stage failure: Task 115 in stage 1087.0 failed 1 times, most recent failure: Lost task 115.0 in stage 1087.0 (TID 4754) (e68dbfc8961f executor driver): software.amazon.awssdk.core.exception.SdkClientException: Received an UnknownHostException when attempting to interact with a service. See cause for the exact endpoint that is failing to resolve. If this is happening on an endpoint that previously worked, there may be a network connectivity issue or your DNS cache could be storing endpoints for too long.

In [38]:
spark.sql("SELECT player_gamertag, match_id, player_total_kills FROM bucketed_result order by 3 desc").show()

+---------------+--------------------+------------------+
|player_gamertag|            match_id|player_total_kills|
+---------------+--------------------+------------------+
|   gimpinator14|acf0e47e-20ac-4b1...|               109|
|   gimpinator14|acf0e47e-20ac-4b1...|               109|
|   gimpinator14|acf0e47e-20ac-4b1...|               109|
|   gimpinator14|acf0e47e-20ac-4b1...|               109|
|   gimpinator14|acf0e47e-20ac-4b1...|               109|
|   gimpinator14|acf0e47e-20ac-4b1...|               109|
|   gimpinator14|acf0e47e-20ac-4b1...|               109|
|   gimpinator14|acf0e47e-20ac-4b1...|               109|
|   gimpinator14|acf0e47e-20ac-4b1...|               109|
|   gimpinator14|acf0e47e-20ac-4b1...|               109|
|   gimpinator14|acf0e47e-20ac-4b1...|               109|
|   gimpinator14|acf0e47e-20ac-4b1...|               109|
|   gimpinator14|acf0e47e-20ac-4b1...|               109|
|   gimpinator14|acf0e47e-20ac-4b1...|               109|
|   gimpinator

In [48]:
// Which player averages the most kills per game?
//val avgKillPerGame = bucketedJoin.groupBy($"player_gamertag")
//  .agg(avg($"player_total_kills").as("avg_kills_per_match"))
//  .orderBy(avg($"player_total_kills").desc)

//Which playlist gets played the most?
val playlistCount = bucketedJoin.groupBy($"playlist_id")
  .agg(countDistinct($"match_id").as("total_games_played"))
  .orderBy(countDistinct($"match_id").desc)


playlistCount.show()


org.apache.spark.SparkException:  Job aborted due to stage failure: Task 0 in stage 1082.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1082.0 (TID 4629) (e68dbfc8961f executor driver): software.amazon.awssdk.core.exception.SdkClientException: Received an UnknownHostException when attempting to interact with a service. See cause for the exact endpoint that is failing to resolve. If this is happening on an endpoint that previously worked, there may be a network connectivity issue or your DNS cache could be storing endpoints for too long.

In [39]:
val match_detailsBucketed = spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/match_details.csv")

val matchesBucketed = spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/matches.csv")
                        
val me_ma_plBucketed = spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/medals_matches_players.csv")

val medalsBucketed = spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/medals.csv")
                        
val mapsBucketed = spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/maps.csv")
                        
matchesBucketed.show()

+--------------------+--------------------+------------+--------------------+--------------------+-------------+-------------------+--------------+---------+--------------------+
|            match_id|               mapid|is_team_game|         playlist_id|     game_variant_id|is_match_over|    completion_date|match_duration|game_mode|      map_variant_id|
+--------------------+--------------------+------------+--------------------+--------------------+-------------+-------------------+--------------+---------+--------------------+
|11de1a94-8d07-416...|c7edbf0f-f206-11e...|        true|f72e0ef0-7c4a-430...|1e473914-46e4-408...|         true|2016-02-22 00:00:00|          NULL|     NULL|                NULL|
|d3643e71-3e51-43e...|cb914b9e-f206-11e...|       false|d0766624-dbd7-453...|257a305e-4dd3-41f...|         true|2016-02-14 00:00:00|          NULL|     NULL|                NULL|
|d78d2aae-36e4-48a...|c7edbf0f-f206-11e...|        true|f72e0ef0-7c4a-430...|1e473914-46e4-408...|       

match_detailsBucketed: org.apache.spark.sql.DataFrame = [match_id: string, player_gamertag: string ... 34 more fields]
matchesBucketed: org.apache.spark.sql.DataFrame = [match_id: string, mapid: string ... 8 more fields]
me_ma_plBucketed: org.apache.spark.sql.DataFrame = [match_id: string, player_gamertag: string ... 2 more fields]
medalsBucketed: org.apache.spark.sql.DataFrame = [medal_id: bigint, sprite_uri: string ... 10 more fields]
mapsBucketed: org.apache.spark.sql.DataFrame = [mapid: string, name: string ... 1 more field]


In [8]:
spark.sql("SELECT player_gamertag, avg(player_total_kills) over (partition by player_gamertag) as avg_kill FROM bucketed_result LIMIT 10").show()

+---------------+--------+
|player_gamertag|avg_kill|
+---------------+--------+
|  A 29 Delivery|    11.0|
|  A 29 Delivery|    11.0|
|  A 29 Delivery|    11.0|
|    A BOOTY TAP|     2.0|
|    A BOOTY TAP|     2.0|
|    A BOOTY TAP|     2.0|
|    A Baby Lynx|    12.0|
|    A Baby Lynx|    12.0|
|    A Baby Lynx|    12.0|
| A Blind Kid 8P|    29.0|
+---------------+--------+

